In [28]:
import xml.etree.ElementTree as ET
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
import numpy as np

data = []
target = []
categories = ["current", "past", "non", "unknown"]

In [22]:
def parse(filename, status):
    tree = ET.parse(filename)
    #this filename can be changed to accommodate different smoker files
    root = tree.getroot()

    for element in root.findall('RECORD'):
        text = str(element.find('TEXT').text)[50:]
        data.append(text)
        if status:
            s= str(element.find("SMOKING").attrib['STATUS'])
            if "UNK" in s:
                target.append(4)
            elif "NON" in s:
                target.append(3)
            elif "PAST" in s:
                target.append(2)
            else:
                target.append(1)

In [23]:
parse('smokers_surrogate_train_all_version2.xml', True)

In [24]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(data)
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
clf = MultinomialNB().fit(X_train_tfidf, target)

In [25]:
parse('smokers_surrogate_test_all_groundtruth_version2.xml', True)
X_new_counts = count_vect.transform(data)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)
predicted = clf.predict(X_new_tfidf)

In [26]:
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB()),
])

In [27]:
print(np.mean(predicted == target))

0.627490039841


In [29]:
text_clf_svm = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()), 
                     ('clf', SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, n_iter=5, random_state=42)),
])

In [ ]:
parse('smokers_surrogate_train_all_version2.xml', True)
_ = text_clf_svm.fit(data, target)
parse('smokers_surrogate_test_all_groundtruth_version2.xml', True)
predicted = text_clf.predict(docs_test)
np.mean(predicted == twenty_test.target)